In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import geopandas as gpd
import pandas as pd
import os
import urllib
import zipfile
import re
from descartes import PolygonPatch
from pysal.model import mgwr
import pysal.lib
import libpysal as ps
from libpysal.weights.contiguity import Queen
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
# conda install mgwr
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap, truncate_colormap
import warnings
warnings.filterwarnings("ignore")

In [51]:
from shapely import wkt


In [53]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

In [2]:
# Configure the download
url  = 'https://github.com/cusp-london/Spatial-Data-Analysis/blob/master/housing_in_london.csv?raw=true'
path = os.path.join("data","housing_in_london.csv")

# Download
r    = urllib.request.urlretrieve(url, path)


In [3]:
df = pd.read_csv('data/LSOA_Data (1).csv')

In [18]:
df.head()

,LSOA11NM,LAD11CD,LAD11NM,POPDEN,HHOLDS,AVHHOLDSZ,geometry,GreenspaceArea,Owned,SharedOwnshp,...,SumPrice,MeanPrice,MedianPrice,EntireHome,PrivateRoom,SharedRoom,SmallHost,MultiHost,PropertyCount,Area
0,City of London 001A,E09000001,City of London,112.9,876,1.7,"POLYGON ((532105.0919998939 182011.23, 532162....",3679.885393,533.0,3.0,...,2589.0,152.294118,146.0,14.0,1.0,2.0,11.0,6.0,17.0,133320.768859
1,City of London 001B,E09000001,City of London,62.9,830,1.7,"POLYGON ((532746.8130000263 181786.891000028, ...",0.000000,527.0,8.0,...,2697.0,149.833333,117.5,12.0,5.0,1.0,13.0,5.0,18.0,226191.273063
2,City of London 001C,E09000001,City of London,227.7,817,1.5,"POLYGON ((532135.1449999654 182198.1190000199,...",0.000000,327.0,1.0,...,1049.0,87.416667,89.5,7.0,5.0,0.0,12.0,0.0,12.0,57302.966528
3,City of London 001E,E09000001,City of London,52.0,467,2.1,"POLYGON ((533807.9460001207 180767.7700000888,...",2680.374365,46.0,0.0,...,5360.0,105.098039,95.0,36.0,14.0,1.0,30.0,21.0,51.0,190738.760508
4,Barking and Dagenham 016A,E09000002,Barking and Dagenham,116.2,543,3.1,"POLYGON ((545122.048999952 184314.931000118, 5...",0.000000,345.0,0.0,...,30.0,30.000000,30.0,0.0,1.0,0.0,1.0,0.0,1.0,144195.846859


In [11]:
df.count()

LSOA11CD          4835
LSOA11NM          4835
MSOA11CD          4835
MSOA11NM          4835
LAD11CD           4835
LAD11NM           4835
RGN11CD           4835
RGN11NM           4835
USUALRES          4835
HHOLDRES          4835
COMESTRES         4835
POPDEN            4835
HHOLDS            4835
AVHHOLDSZ         4835
geometry          4835
GreenspaceArea    4835
RoadsArea         4835
Owned             4835
SharedOwnshp      4835
SocialRented      4835
PrivateRented     4835
RentFree          4835
ORroomsM1         4835
ORbedsM1          4835
RoomsHH           4835
BedsHH            4835
White             4835
MultiEthnic       4835
Asian             4835
Black             4835
OtherEthnic       4835
MedianIncome      4835
SumPrice          4835
MeanPrice         4835
MedianPrice       4835
EntireHome        4835
PrivateRoom       4835
SharedRoom        4835
SmallHost         4835
MultiHost         4835
PropertyCount     4835
Area              4835
dtype: int64

In [13]:
# Count the data based on the town's name
df['Owned'].value_counts()

394.0    23
397.0    21
421.0    20
338.0    19
488.0    18
         ..
5.0       1
604.0     1
18.0      1
647.0     1
672.0     1
Name: Owned, Length: 597, dtype: int64

In [6]:
# drop those irrelevant columns for cleanliess
df.drop(['MSOA11CD', 'MSOA11NM', 'RGN11CD', 'RGN11NM', 'USUALRES', 'HHOLDRES' ,'AVHHOLDSZ', 'COMESTRES' ], axis=1, inplace=True)

In [7]:
# Create a correlation matrix here using pandas
corr= df.corr()

# style it
corr.style.background_gradient(cmap='coolwarm')

,POPDEN,HHOLDS,GreenspaceArea,Owned,SharedOwnshp,SocialRented,PrivateRented,RentFree,ORroomsM1,ORbedsM1,RoomsHH,BedsHH,White,MultiEthnic,Asian,Black,OtherEthnic,MedianIncome,SumPrice,MeanPrice,MedianPrice,EntireHome,PrivateRoom,SharedRoom,SmallHost,MultiHost,PropertyCount,Area
POPDEN,1.000000,0.204152,-0.187109,-0.544204,0.116787,0.423196,0.342165,0.121374,0.554999,0.497563,-0.548674,-0.508166,-0.259837,0.313679,0.101198,0.288523,0.312515,-0.130707,0.257522,0.189150,0.204351,0.303654,0.354625,0.092931,0.370433,0.239287,0.349849,-0.371702
HHOLDS,0.204152,1.000000,0.000367,0.025557,0.342939,0.346272,0.662475,0.372356,0.605094,0.324670,-0.500005,-0.559564,0.451432,0.480293,-0.144942,0.161417,0.226435,0.102621,0.392985,0.222461,0.234974,0.419479,0.394819,0.110495,0.470115,0.298959,0.441860,-0.035738
GreenspaceArea,-0.187109,0.000367,1.000000,0.079326,-0.025071,-0.048292,-0.033934,0.048015,-0.077396,-0.088782,0.105472,0.090587,0.078587,-0.050847,-0.032221,-0.064580,-0.036531,0.106265,0.012261,0.061482,0.065162,-0.004853,-0.008278,-0.002179,-0.008100,-0.003212,-0.006737,0.366683
Owned,-0.544204,0.025557,0.079326,1.000000,-0.222751,-0.744663,-0.187970,-0.132686,-0.642870,-0.676812,0.680892,0.609733,0.567561,-0.392735,-0.137078,-0.543058,-0.347853,0.467369,-0.216154,-0.111639,-0.127095,-0.248975,-0.353647,-0.110632,-0.327625,-0.224456,-0.315212,0.231172
SharedOwnshp,0.116787,0.342939,-0.025071,-0.222751,1.000000,0.264076,0.206675,0.047841,0.421340,0.300195,-0.333908,-0.311124,-0.021515,0.298092,0.077112,0.234686,0.140968,-0.157363,0.111287,0.025933,0.035693,0.137628,0.205348,0.044806,0.218119,0.078409,0.177770,-0.038462
SocialRented,0.423196,0.346272,-0.048292,-0.744663,0.264076,1.000000,-0.012000,0.093736,0.599427,0.611350,-0.634841,-0.620686,-0.190871,0.516449,-0.094766,0.617462,0.251896,-0.509113,0.152761,0.046493,0.060153,0.186346,0.315881,0.089788,0.296387,0.143205,0.258120,-0.133799
PrivateRented,0.342165,0.662475,-0.033934,-0.187970,0.206675,-0.012000,1.000000,0.422603,0.667964,0.380210,-0.556176,-0.560015,0.098994,0.343712,0.105818,0.026516,0.332777,0.220515,0.498181,0.321743,0.336311,0.527571,0.461743,0.142465,0.539664,0.417880,0.540971,-0.147862
RentFree,0.121374,0.372356,0.048015,-0.132686,0.047841,0.093736,0.422603,1.000000,0.306717,0.133188,-0.247462,-0.286897,0.059305,0.135884,0.007161,-0.014937,0.354530,0.254909,0.494217,0.348922,0.357006,0.430730,0.216014,0.076921,0.301052,0.386627,0.371130,-0.008146
ORroomsM1,0.554999,0.605094,-0.077396,-0.642870,0.421340,0.599427,0.667964,0.306717,1.000000,0.803630,-0.819112,-0.773349,-0.235215,0.566662,0.211078,0.469992,0.483522,-0.300809,0.433936,0.195452,0.218600,0.493336,0.526230,0.171309,0.553330,0.413451,0.547889,-0.197441
ORbedsM1,0.497563,0.324670,-0.088782,-0.676812,0.300195,0.611350,0.380210,0.133188,0.803630,1.000000,-0.651843,-0.580440,-0.503801,0.562136,0.438463,0.671051,0.484667,-0.589948,0.106105,-0.006800,0.000643,0.143718,0.303815,0.110864,0.241389,0.152857,0.226588,-0.190159


In [8]:
#find design matrix for linear regression model using 'rating' as response variable 
y, X = dmatrices('Owned ~ White+Black+Asian+MultiEthnic+OtherEthnic', data=df, return_type='dataframe')

#calculate VIF for each explanatory variable
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['variable'] = X.columns

#view VIF for each explanatory variable 
vif

,VIF,variable
0,47.481649,Intercept
1,2.752004,White
2,2.751748,Black
3,2.029675,Asian
4,2.104381,MultiEthnic
5,1.213443,OtherEthnic


In [9]:
# First, you need to prepare the dataset so it can be processed in pysal
ownership = df['Owned']
# OLS class requires data to be passed as numpy array
y = np.array(ownership)
# PySAL requires your dependent variable to be nx1 numpy array
y.shape = (len(ownership),1)

In [10]:
# Now prepare your independent variables
X = []
X.append(df['White'])
X.append(df['Black'])
X.append(df['Asian'])
X.append(df['MultiEthnic'])
X.append(df['OtherEthnic'])
# PySAL requires your independent variable to be a 2D nxj numpy array where j is the number of independent variables
X=np.array(X).T

In [11]:
# Check if your independent variables are in the right format
print(X.shape)

(4835, 5)


In [12]:
# Your code for OLS regression here using pysal.model.spreg.OLS
ols = pysal.model.spreg.OLS(y, X, name_y='Owned',\
          name_x=['White','Black','Asian', 'MultiEthnic', 'OtherEthnic'],\
          name_ds='London', white_test=True)
print(ols.summary)
                        

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :      London
Weights matrix      :        None
Dependent Variable  :       Owned                Number of Observations:        4835
Mean dependent var  :    325.9994                Number of Variables   :           6
S.D. dependent var  :    135.2996                Degrees of Freedom    :        4829
R-squared           :      0.5074
Adjusted R-squared  :      0.5069
Sum squared residual:43589646.936                F-statistic           :    994.8647
Sigma-square        :    9026.640                Prob(F-statistic)     :           0
S.E. of regression  :      95.009                Log likelihood        :  -28876.000
Sigma-square ML     :    9015.439                Akaike info criterion :   57764.000
S.E of regression ML:     94.9497                Schwarz criterion     :   57802.902

-----------------------------------------------------------------------------

In [19]:
# We can do this using statsmodel as well, just as a comparison

x = ['White','Black','Asian', 'MultiEthnic', 'OtherEthnic'] # define input/independant variables
X = df.loc[:, x].dropna()
X = sm.add_constant(X)
y = np.array(df['Owned']) # define output/dependant variable
# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     994.9
Date:                Fri, 26 Nov 2021   Prob (F-statistic):               0.00
Time:                        20:08:33   Log-Likelihood:                -28876.
No. Observations:                4835   AIC:                         5.776e+04
Df Residuals:                    4829   BIC:                         5.780e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         114.4360      9.415     12.154      

In [26]:
#Prepare dataset inputs
g_y = df['Owned'].values.reshape((-1,1))
g_X = df[['White','Black','Asian', 'MultiEthnic', 'OtherEthnic']].values
u = X
v = y
g_coords = list(zip(u,v))

# Standardised our data to have mean of 0 and standard deviation of 1
g_X = (g_X - g_X.mean(axis=0)) / g_X.std(axis=0)

g_y = g_y.reshape((-1,1))

g_y = (g_y - g_y.mean(axis=0)) / g_y.std(axis=0)

In [20]:
X

,const,White,Black,Asian,MultiEthnic,OtherEthnic
0,1.0,1238.0,11.0,128.0,54.0,34.0
1,1.0,1274.0,4.0,95.0,54.0,9.0
2,1.0,1055.0,45.0,168.0,55.0,23.0
3,1.0,506.0,100.0,274.0,59.0,46.0
4,1.0,557.0,177.0,861.0,58.0,50.0
...,...,...,...,...,...,...
4830,1.0,618.0,423.0,186.0,81.0,44.0
4831,1.0,593.0,1154.0,144.0,121.0,26.0
4832,1.0,881.0,613.0,235.0,82.0,34.0
4833,1.0,748.0,837.0,112.0,85.0,38.0


In [27]:
# Select bandwidth for kernel
bw = mgwr.sel_bw.Sel_BW(g_coords, 
                  g_y, # Independent variable
                  g_X, # Dependent variable
                  fixed=False, # True for fixed bandwidth and false for adaptive bandwidth
                  spherical=True) # Spherical coordinates (long-lat) or projected coordinates

In [28]:
# Calculate the optimum bandwidth for our local regression?
bw.search()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function sub>) found for signature:
 
 >>> sub(array([unichr x 32], 1d, A), [unichr x 32])
 
There are 12 candidate implementations:
[1m    - Of which 10 did not match due to:
    Overload of function 'sub': File: <numerous>: Line N/A.
      With argument(s): '(array([unichr x 32], 1d, A), [unichr x 32])':[0m
[1m     No match.[0m
[1m    - Of which 2 did not match due to:
    Operator Overload in function 'sub': File: unknown: Line unknown.
      With argument(s): '(array([unichr x 32], 1d, A), [unichr x 32])':[0m
[1m     No match for registered cases:
      * (int64, int64) -> int64
      * (int64, uint64) -> int64
      * (uint64, int64) -> int64
      * (uint64, uint64) -> uint64
      * (float32, float32) -> float32
      * (float64, float64) -> float64
      * (complex64, complex64) -> complex64
      * (complex128, complex128) -> complex128[0m
[

In [43]:

pip install --pre numba

Note: you may need to restart the kernel to use updated packages.


In [38]:
df2 = pd.read_csv('data/LSOA_Data (1).csv')

,LSOA11NM,LAD11CD,LAD11NM,POPDEN,HHOLDS,geometry,GreenspaceArea,Owned,SharedOwnshp,SocialRented,...,SumPrice,MeanPrice,MedianPrice,EntireHome,PrivateRoom,SharedRoom,SmallHost,MultiHost,PropertyCount,Area
0,City of London 001A,E09000001,City of London,112.9,876,"POLYGON ((532105.0919998939 182011.23, 532162....",3679.885393,533.0,3.0,41.0,...,2589.0,152.294118,146.0,14.0,1.0,2.0,11.0,6.0,17.0,133320.768859
1,City of London 001B,E09000001,City of London,62.9,830,"POLYGON ((532746.8130000263 181786.891000028, ...",0.000000,527.0,8.0,48.0,...,2697.0,149.833333,117.5,12.0,5.0,1.0,13.0,5.0,18.0,226191.273063
2,City of London 001C,E09000001,City of London,227.7,817,"POLYGON ((532135.1449999654 182198.1190000199,...",0.000000,327.0,1.0,295.0,...,1049.0,87.416667,89.5,7.0,5.0,0.0,12.0,0.0,12.0,57302.966528
3,City of London 001E,E09000001,City of London,52.0,467,"POLYGON ((533807.9460001207 180767.7700000888,...",2680.374365,46.0,0.0,312.0,...,5360.0,105.098039,95.0,36.0,14.0,1.0,30.0,21.0,51.0,190738.760508
4,Barking and Dagenham 016A,E09000002,Barking and Dagenham,116.2,543,"POLYGON ((545122.048999952 184314.931000118, 5...",0.000000,345.0,0.0,18.0,...,30.0,30.000000,30.0,0.0,1.0,0.0,1.0,0.0,1.0,144195.846859


In [39]:
df2.head()

,LSOA11CD,LSOA11NM,MSOA11CD,MSOA11NM,LAD11CD,LAD11NM,RGN11CD,RGN11NM,USUALRES,HHOLDRES,...,SumPrice,MeanPrice,MedianPrice,EntireHome,PrivateRoom,SharedRoom,SmallHost,MultiHost,PropertyCount,Area
0,E01000001,City of London 001A,E02000001,City of London 001,E09000001,City of London,E12000007,London,1465,1465,...,2589.0,152.294118,146.0,14.0,1.0,2.0,11.0,6.0,17.0,133320.768859
1,E01000002,City of London 001B,E02000001,City of London 001,E09000001,City of London,E12000007,London,1436,1436,...,2697.0,149.833333,117.5,12.0,5.0,1.0,13.0,5.0,18.0,226191.273063
2,E01000003,City of London 001C,E02000001,City of London 001,E09000001,City of London,E12000007,London,1346,1250,...,1049.0,87.416667,89.5,7.0,5.0,0.0,12.0,0.0,12.0,57302.966528
3,E01000005,City of London 001E,E02000001,City of London 001,E09000001,City of London,E12000007,London,985,985,...,5360.0,105.098039,95.0,36.0,14.0,1.0,30.0,21.0,51.0,190738.760508
4,E01000006,Barking and Dagenham 016A,E02000017,Barking and Dagenham 016,E09000002,Barking and Dagenham,E12000007,London,1703,1699,...,30.0,30.000000,30.0,0.0,1.0,0.0,1.0,0.0,1.0,144195.846859


TypeError: Input must be valid geometry objects: POLYGON ((532105.0919998939 182011.23, 532162.4910001159 181867.7629999601, 532248.3160000765 181895.3249998323, 532282.6300000258 181906.4960000554, 532308.6079999561 181915.5200001549, 532303.4919998596 181814.1099998362, 532251.1259998521 181720.0000001175, 532267.7280001113 181643.780999956, 532213.063999875 181477.2030001849, 532282.2499999444 181460.4999999297, 532248.2490000051 181332.0360001744, 532227.687999856 181278.8749999276, 532173.1249999898 181263.453, 532074.3749998672 181338.2970001521, 532080.2499999747 181456.7499999724, 531948.313000044 181471.96900013, 531951.8749998901 181548.0000001803, 531985.8750000516 181583.4999998761, 532052.4999999914 181600.3909998685, 532064.3750001056 181561.5940001571, 532095.5629998141 181577.3510000679, 532077.7050000296 181797.669000078, 532069.8130000822 181825.9059999275, 532021.1880001619 181803.8910001202, 532022.3739999083 181893.4690001195, 532082.8759998722 181911.7809999972, 5

In [43]:
db_new = df2.fillna(0)

In [45]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4835 entries, 0 to 4834
Data columns (total 42 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   LSOA11CD        4835 non-null   object 
 1   LSOA11NM        4835 non-null   object 
 2   MSOA11CD        4835 non-null   object 
 3   MSOA11NM        4835 non-null   object 
 4   LAD11CD         4835 non-null   object 
 5   LAD11NM         4835 non-null   object 
 6   RGN11CD         4835 non-null   object 
 7   RGN11NM         4835 non-null   object 
 8   USUALRES        4835 non-null   int64  
 9   HHOLDRES        4835 non-null   int64  
 10  COMESTRES       4835 non-null   int64  
 11  POPDEN          4835 non-null   float64
 12  HHOLDS          4835 non-null   int64  
 13  AVHHOLDSZ       4835 non-null   float64
 14  geometry        4835 non-null   object 
 15  GreenspaceArea  4835 non-null   float64
 16  RoadsArea       4835 non-null   float64
 17  Owned           4835 non-null   f

In [52]:
df2['geometry'] = df2['geometry'].apply(wkt.loads)

In [53]:
db_geo = gpd.GeoDataFrame(df2, geometry='geometry')
db_geo.to_file(driver='ESRI Shapefile', filename='SDSGroup.shp')

In [55]:
#find design matrix for linear regression model using 'rating' as response variable 
y, X = dmatrices('Owned ~ White+Black+Asian+MultiEthnic+OtherEthnic', data=df2, return_type='dataframe')

#calculate VIF for each explanatory variable
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['variable'] = X.columns

#view VIF for each explanatory variable 
vif

,VIF,variable
0,47.481649,Intercept
1,2.752004,White
2,2.751748,Black
3,2.029675,Asian
4,2.104381,MultiEthnic
5,1.213443,OtherEthnic


In [56]:
# First, you need to prepare the dataset so it can be processed in pysal
ownership = df2['Owned']
# OLS class requires data to be passed as numpy array
y = np.array(ownership)
# PySAL requires your dependent variable to be nx1 numpy array
y.shape = (len(ownership),1)

In [59]:
# Now prepare your independent variables
X = []
X.append(df2['White'])
X.append(df2['Black'])
X.append(df2['Asian'])
X.append(df2['MultiEthnic'])
X.append(df2['OtherEthnic'])
# PySAL requires your independent variable to be a 2D nxj numpy array where j is the number of independent variables
X=np.array(X).T

In [60]:
# Your code for OLS regression here using pysal.model.spreg.OLS
ols = pysal.model.spreg.OLS(y, X, name_y='Owned',\
          name_x=['White','Black','Asian', 'MultiEthnic', 'OtherEthnic'],\
          name_ds='London', white_test=True)
print(ols.summary)
                        

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :      London
Weights matrix      :        None
Dependent Variable  :       Owned                Number of Observations:        4835
Mean dependent var  :    325.9994                Number of Variables   :           6
S.D. dependent var  :    135.2996                Degrees of Freedom    :        4829
R-squared           :      0.5074
Adjusted R-squared  :      0.5069
Sum squared residual:43589646.936                F-statistic           :    994.8647
Sigma-square        :    9026.640                Prob(F-statistic)     :           0
S.E. of regression  :      95.009                Log likelihood        :  -28876.000
Sigma-square ML     :    9015.439                Akaike info criterion :   57764.000
S.E of regression ML:     94.9497                Schwarz criterion     :   57802.902

-----------------------------------------------------------------------------

In [61]:
# We can do this using statsmodel as well, just as a comparison

x = ['White','Black','Asian', 'MultiEthnic', 'OtherEthnic'] # define input/independant variables
X = df2.loc[:, x].dropna()
X = sm.add_constant(X)
y = np.array(df2['Owned']) # define output/dependant variable
# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     994.9
Date:                Fri, 26 Nov 2021   Prob (F-statistic):               0.00
Time:                        20:39:01   Log-Likelihood:                -28876.
No. Observations:                4835   AIC:                         5.776e+04
Df Residuals:                    4829   BIC:                         5.780e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         114.4360      9.415     12.154      

In [75]:
#Prepare dataset inputs
g_y = df2['Owned'].values.reshape((-1,1))
g_X = df2[['White','Black','Asian', 'MultiEthnic', 'OtherEthnic']].values
u = df2['X']
v = df2['Y']
g_coords = list(zip(u,v))

# Standardised our data to have mean of 0 and standard deviation of 1
g_X = (g_X - g_X.mean(axis=0)) / g_X.std(axis=0)

g_y = g_y.reshape((-1,1))

g_y = (g_y - g_y.mean(axis=0)) / g_y.std(axis=0)

KeyError: 'X'

In [79]:
df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
df2 = gpd.GeoDataFrame(df2, geometry='geometry')

TypeError: Expected bytes, got Polygon